In [41]:
import pandas as pd
import numpy as np
import ast

def convert_to_floats(string_list):
    nested_list = ast.literal_eval(string_list)
    return [item[0] for item in nested_list]


df = pd.read_csv('data.csv')



df = df[df['type'] == 'pk']

df['probability']

df['probability'] = df['probability'].apply(convert_to_floats)

df


TypeError: 'float' object is not subscriptable

In [ ]:
try:
    df = pd.read_csv('data.csv')
    df = df.sort_values(['N', 'tau', 'noise'])
except FileNotFoundError:
    df = pd.DataFrame(columns=['N', 'tau', 'type', 'noise','probability', 'mean', 
                            'second_moment', 'third_moment', 'fourth_moment',
                            'variance', 'skewness', 'kurtosis'])

# Check if data[N][tau][w] is in DataFrame
if (not df.empty) and (not df[(df['N'] == N) & (df['tau'] == tau) & (df['noise'] == w)].empty):
    spesific_df = df[(df['N'] == N) & (df['tau'] == tau) & (df['noise'] == w)]
    ks = k_f(np.arange(0, N/2), N)
    d_vals = np.arange(0,N+1,2)
    if spesific_df[spesific_df['type'] == 'pk'].empty:
        pks_numeric = calk_noisy_pk(ks, tau, w)
        pk_data_df = pd.DataFrame({'probability': [pks_numeric.tolist()], 'N': [N], 'tau': [tau], 'type': ['pk'], 'noise': [w]})
        with lock:
            df = pd.concat([df, pk_data_df])
            df.to_csv('data.csv', index=False)
    else: 
        pks_numeric = np.array(spesific_df[spesific_df['type'] == 'pk']['probability'].values[0])
    
    if spesific_df[spesific_df['type'] == 'numeric'].empty:
        num_probability_mass_function = calc_kink_probabilities(pks_numeric, d_vals)
        num_probability_mass_function = np.where(num_probability_mass_function < epsilon, 0, num_probability_mass_function)
        num_mask = (np.roll(num_probability_mass_function, 0) == 0) & (np.roll(num_probability_mass_function, -2) == 0)
        num_mask[-1] = False
        num_mask[-2] = False
        num_probability_mass_function[np.roll(num_mask, 1)] = 0
        num_cumulants = calculate_cumulants(num_probability_mass_function, d_vals)
        with lock:
            num_data_df = pd.DataFrame({**num_cumulants, 'probability': [num_probability_mass_function.tolist()], 'N': [N], 'tau': [tau], 'type': ['numeric'], 'noise': [w]})
            df = pd.concat([df, num_data_df])
            df.to_csv('data.csv', index=False)
    else:
        num_probability_mass_function = np.array(spesific_df[spesific_df['type'] == 'numeric']['probability'].values[0])
        
    if spesific_df[spesific_df['type'] == 'thermal'].empty:
        d_num = D_func(num_probability_mass_function)
        therm_probability_mass_function = thermal_prob(d_num, N)
        therm_probability_mass_function = np.where(therm_probability_mass_function < epsilon, 0, therm_probability_mass_function)
        therm_mask = (np.roll(therm_probability_mass_function, 0) == 0) & (np.roll(therm_probability_mass_function, -2) == 0)
        therm_mask[-1] = False
        therm_mask[-2] = False
        therm_probability_mass_function[np.roll(therm_mask, 1)] = 0
        therm_cumulants = calculate_cumulants(therm_probability_mass_function, d_vals)
        with lock:
            therm_data_df = pd.DataFrame({**therm_cumulants, 'probability': [therm_probability_mass_function.tolist()], 'N': [N], 'tau': [tau], 'type': ['thermal'], 'noise': [w]})
            df = pd.concat([df, therm_data_df])
            df.to_csv('data.csv', index=False)